# 将检索模块和文本聚类模块进行封装

In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch

In [2]:
import mysql.connector

# ES检索模块重构为一个对象

In [3]:
columns = ['C_ID',
 'C_comment_id',
 'C_comment_user_id',
 'C_comment_user_nick_name',
 'C_content',
 'C_weibo_url',
 'C_like_num',
 'C_created_at',
 'C_crawl_time']

In [4]:
','.join(columns)

'C_ID,C_comment_id,C_comment_user_id,C_comment_user_nick_name,C_content,C_weibo_url,C_like_num,C_created_at,C_crawl_time'

In [44]:
class esWeiboCommentRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esWeiboCommentRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esWeiboCommentRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'weibo-comment-index'

    def do_search(self, wordQuery, textQuery, volume):
        '''
        do_search方法执行具体检索过程
        wordQuery 本应为查询对应微博时所用的检索词 此处暂时不用 目前暂时只检索微博评论数据后续再修改对象为
        
        volume为每次检索返回的数目
        '''
        queryBody = {
          "query": {
            "match": {
              "C_content": textQuery
            }
          },
          "from": 0,
          "size": volume,
          "sort": [],
          "aggs": {}
        }
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'C_ID,C_comment_id,C_comment_user_id,C_comment_user_nick_name,C_content,\
        C_weibo_url,C_like_num,C_created_at,C_crawl_time'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs
    def Retrieval(self, wordQuery, textQuery, volume):
        result = self.do_search(wordQuery, textQuery, volume)
        docs = self.format_search(result)
        return docs

In [45]:
eee = esWeiboCommentRetrieval(host='10.8.128.205',port=49200,)

In [46]:
result= eee.Retrieval( wordQuery='我', textQuery='我',volume=100)

# 客户端对于后端数据接口的访问

In [48]:
import requests

body = {'wordQuery':'我',
        'textQuery':'垃圾分类',
        'volume':1000}

baseUrl = 'http://10.8.128.205:29280/Lawbda/dataWare/1.0.0/weibo/search'

docs = requests.get(baseUrl,params=body)

In [49]:
docs.json()[0]

{'C_ID': 70125,
 'C_comment_id': 'C_4391845553021738',
 'C_comment_user_id': '6079113604',
 'C_comment_user_nick_name': '安心的温柔866',
 'C_content': '#杨洋 垃圾分类# #垃圾分类挑战#  杨洋 垃圾分类  #杨洋[超话]# @杨洋',
 'C_crawl_time': '1970-01-01T00:00:02',
 'C_created_at': '2019-07-08T18:17:00',
 'C_like_num': 1,
 'C_weibo_url': 'https://weibo.com/1625035922/HCwSG4b6Q'}

In [56]:
def get_hot_topic_and_weibos():
    """
    获取热门话题和话题对应的热门微博，每个话题封装成一个字典
    :return:  字典列表，[one_topic_dct{topic, content, nick_name, ...}]
    """
    coon = mysql.connector.connect(user='weibo_user', password='ir@DUT#weibo', host="10.8.128.205",
                                   port='23306')
    cursor = coon.cursor(buffered=True)
    cursor.execute('use weiboDB;')

    ##  热门话题，这里先给定，以后完善
    hot_topics = ['#垃圾分类#', '#韩商言借钱#', '#被辱骂初一女生家长原谅班主任#', '#赵丽颖产后现身#', '#李荣浩成功求婚杨丞琳#',
                  '#2018结婚率创十年新低#', '#中国花协推荐牡丹为国花#', '#2018结婚率创十年新低#', '#香港警方拘捕47人#', '#精神病患是否可坐飞机#', ]

    hot_topics_msg_lst = []
    for hot_topic in hot_topics:
        # 挑选每个热门话题的热门微博，这里只选择评论最多的一条
        # 从数据库中根据 W_search_query 的值查找，返回5列内容
        sql_base = 'select W_content, W_nick_name, W_repost_num, W_comment_num, W_like_num \
                   from weibo_table where W_search_query="{}";'
        sql_str = sql_base.format(hot_topic)
        cursor.execute(sql_str)
        items = cursor.fetchall()

        hot_weibo = sorted(items, key=lambda x: x[3], reverse=True)[0]
        hot_weibo_dct = {'topic': hot_topic,
                         'content': hot_weibo[0], 'nick_name': hot_weibo[1],
                         'repost_num': hot_weibo[2], 'comment_num': hot_weibo[3],
                         'like_num': hot_weibo[4]}
        hot_topics_msg_lst.append(hot_weibo_dct)

    coon.close()
    return hot_topics_msg_lst

In [57]:
get_hot_topic_and_weibos()

[{'topic': '#垃圾分类#',
  'content': '哈哈哈哈哈哈哈哈哈哈哈哈哈半夜给我笑死，敲黑板，来记一下知识点！！ 自己家狗拉的屎！都给我捡起来！拿回家冲马桶！然后包屎的垃圾袋，纸巾，都是干垃圾！ #垃圾分类# (via 朋友圈解说小楼)',
  'nick_name': '芋圆吃饱没',
  'repost_num': 4693,
  'comment_num': 4312,
  'like_num': 5246},
 {'topic': '#韩商言借钱#',
  'content': '韩商言求抱抱 韩商言借钱给年年过生日 杨紫李现胡一天《亲爱的热爱的》15、16集预告（东方版）：年年言言和好了，摸头杀【佟年：我不想分手 韩商言：那就不分】；老男人一本正经教年年谈恋爱，抱抱被拒绝[笑cry] 韩商言：你最近是怎么约会的，只是我不太懂。 佟年：你是我的初恋 #韩商言借钱# #李现唱小毛驴# #亲爱的热爱的预告# http://t.cn/AiWuIgPE',
  'nick_name': '星闻揭秘',
  'repost_num': 5243,
  'comment_num': 5222,
  'like_num': 45168},
 {'topic': '#被辱骂初一女生家长原谅班主任#',
  'content': '【陕西#被辱骂初一女生家长原谅班主任# ：其受到的惩罚已足够】目前被辱骂初一女生的家长已原谅当事班主任，认为班主任受到的惩罚已足够，家长表示准备带孩子出去散散心，回归平静生活，尽快扭转孩子情绪。商州区委区政府对其他相关责任人的调查处理正在进行当中，同时，由商州区妇联主席负责的，对女孩心理疏导的工作已经准备就绪，相关专业人员也已准备到位，可以根据家长和孩子意愿随时展开。 @陕西都市快报  http://t.cn/AiWYEC7p[组图共4张]',
  'nick_name': '澎湃新闻',
  'repost_num': 1454,
  'comment_num': 8091,
  'like_num': 8649},
 {'topic': '#赵丽颖产后现身#',
  'content': '#赵丽颖产后现身# 16日，@赵丽颖 身穿蓝紫色长裙和白色帆布鞋现身北京机场，戴着帽子口罩的她包裹严实， 身材苗条腰肢纤细，状态十分